In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

from sklearn.feature_selection import mutual_info_classif

pd.plotting.register_matplotlib_converters()
%matplotlib inline

import duckdb
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import precision_score, roc_auc_score, recall_score, accuracy_score

from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
import featuretools as ft

%load_ext autoreload
%autoreload 2
from help_tool import help_tool, help_visuals, help_stats

pd.set_option('future.no_silent_downcasting', True)

# Setting graph parameters
sns.set_theme(style='whitegrid')
sns.set_palette('husl')

In [30]:
previous_application = help_tool.csv_download(r'Archive\home-credit-default-risk\previous_application.csv')
help_tool.first_look(previous_application)

Dataset has 1670214 observations and 37 features
Columns with all empty values []
Dataset has 0 duplicates


,dtypes,"Null values, Count","Null values, %"
SK_ID_PREV,int64,0,0.0
SK_ID_CURR,int64,0,0.0
NAME_CONTRACT_TYPE,object,0,0.0
AMT_ANNUITY,float64,372235,22.3
AMT_APPLICATION,float64,0,0.0
AMT_CREDIT,float64,1,0.0
AMT_DOWN_PAYMENT,float64,895844,53.6
AMT_GOODS_PRICE,float64,385515,23.1
WEEKDAY_APPR_PROCESS_START,object,0,0.0
HOUR_APPR_PROCESS_START,int64,0,0.0


In [26]:
application_train = help_tool.csv_download(r'Archive\home-credit-default-risk\application_train.csv')[['SK_ID_CURR', 'TARGET']]

In [31]:
previous_application = help_tool.product_combination(previous_application)
previous_application['NAME_CLIENT_TYPE'] = help_tool.client_type_encoding(previous_application, 'NAME_CLIENT_TYPE')
previous_application = help_tool.weekday_encoding(previous_application)
previous_application['NAME_TYPE_SUITE'] = help_tool.accompanied(previous_application, 'NAME_TYPE_SUITE')
application_train = help_tool.convert_flags(application_train)

In [32]:
previous_application['YIELD_GROUP'] = previous_application['NAME_YIELD_GROUP'].apply(lambda x: 1 if 'low_normal' in x.lower() else 1.5 if 'low_action' in x.lower() else 2 if 'middle' in x.lower() else 3 if 'high' in x.lower() else np.nan)

In [34]:
previous_application.drop(columns=[
    'NAME_CASH_LOAN_PURPOSE', 
    'NAME_PORTFOLIO', 'NAME_YIELD_GROUP'
    ], inplace=True)

In [35]:
[previous_application.select_dtypes(include=['object']).columns.tolist()]

['NAME_CONTRACT_TYPE',
 'FLAG_LAST_APPL_PER_CONTRACT',
 'NAME_CONTRACT_STATUS',
 'NAME_PAYMENT_TYPE',
 'CODE_REJECT_REASON',
 'NAME_GOODS_CATEGORY',
 'NAME_PRODUCT_TYPE',
 'CHANNEL_TYPE',
 'NAME_SELLER_INDUSTRY',
 'PRODUCT_PLACE']

In [ ]:
previous_application.dtypes#.filter(like='object')

SK_ID_PREV                       int64
SK_ID_CURR                       int64
NAME_CONTRACT_TYPE              object
AMT_ANNUITY                    float64
AMT_APPLICATION                float64
AMT_CREDIT                     float64
AMT_DOWN_PAYMENT               float64
AMT_GOODS_PRICE                float64
WEEKDAY_APPR_PROCESS_START     float64
HOUR_APPR_PROCESS_START          int64
FLAG_LAST_APPL_PER_CONTRACT     object
NFLAG_LAST_APPL_IN_DAY           int64
RATE_DOWN_PAYMENT              float64
RATE_INTEREST_PRIMARY          float64
RATE_INTEREST_PRIVILEGED       float64
NAME_CONTRACT_STATUS            object
DAYS_DECISION                    int64
NAME_PAYMENT_TYPE               object
CODE_REJECT_REASON              object
NAME_TYPE_SUITE                  int32
NAME_CLIENT_TYPE               float64
NAME_GOODS_CATEGORY             object
NAME_PRODUCT_TYPE               object
CHANNEL_TYPE                    object
SELLERPLACE_AREA                 int64
NAME_SELLER_INDUSTRY     

In [ ]:
help_tool.dummy_columns(previous_application, [])

In [7]:
pos_cash_agg = help_tool.csv_download(r'Archive\home-credit-default-risk\pos_cash_feature_matrix.csv').drop(columns='Unnamed: 0')
credit_card_balance_agg = help_tool.csv_download(r'Archive\home-credit-default-risk\credit_card_balance_feature_matrix.csv').drop(columns='Unnamed: 0')
installments_payments_agg = help_tool.csv_download(r'Archive\home-credit-default-risk\installments_payments_feature_matrix.csv').drop(columns='Unnamed: 0')

# Feature Selection

In [8]:
pos_cash_agg.head()

,SK_ID_PREV,SK_ID_CURR,COUNT(pos_cash),MAX(pos_cash.CNT_INSTALMENT),MAX(pos_cash.CNT_INSTALMENT_FUTURE),MAX(pos_cash.RATIO_DPD),MAX(pos_cash.RATIO_Term_Payed_Left),MAX(pos_cash.SK_DPD),MAX(pos_cash.SK_DPD_DEF),MEAN(pos_cash.CNT_INSTALMENT),...,STD(pos_cash.RATIO_Term_Payed_Left),STD(pos_cash.SK_DPD),SUM(pos_cash.CARD_ACTIVITY),SUM(pos_cash.CNT_INSTALMENT),SUM(pos_cash.CNT_INSTALMENT_FUTURE),SUM(pos_cash.RATIO_Term_Payed_Left),SUM(pos_cash.RISK_PROFILE),SUM(pos_cash.SK_DPD),SUM(pos_cash.SK_DPD_DEF),Duration
0,1387235,153211,35,36.0,36.0,1.0,10.0,0.0,0.0,36.0,...,2.127872,0.0,35.0,1260.0,665.0,82.437668,35.0,0.0,0.0,34.0
1,1716688,405549,13,12.0,12.0,NaN,12.0,0.0,0.0,12.0,...,3.639745,0.0,14.0,156.0,78.0,39.920578,13.0,0.0,0.0,12.0
2,1557122,420239,24,24.0,24.0,NaN,12.0,0.0,0.0,24.0,...,3.008212,0.0,25.0,576.0,280.0,71.500000,24.0,0.0,0.0,23.0
3,1206782,380743,13,12.0,12.0,NaN,8.5,0.0,0.0,12.0,...,3.079178,0.0,14.0,156.0,78.0,37.734499,13.0,0.0,0.0,12.0
4,1076996,328058,25,24.0,24.0,NaN,6.0,0.0,0.0,24.0,...,1.273298,0.0,26.0,600.0,300.0,45.463196,25.0,0.0,0.0,24.0


In [9]:
app_pos_cash_agg = duckdb.query("""
             SELECT A.*, B.*, C.TARGET
             FROM pos_cash_agg A
             LEFT JOIN previous_application B ON A.SK_ID_PREV = B.SK_ID_PREV 
             LEFT JOIN application_train C ON A.SK_ID_CURR = C.SK_ID_CURR 
             WHERE C.SK_ID_CURR IS NOT NULL
             """).to_df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [10]:
app_pos_cash_agg.head()

,SK_ID_PREV,SK_ID_CURR,COUNT(pos_cash),MAX(pos_cash.CNT_INSTALMENT),MAX(pos_cash.CNT_INSTALMENT_FUTURE),MAX(pos_cash.RATIO_DPD),MAX(pos_cash.RATIO_Term_Payed_Left),MAX(pos_cash.SK_DPD),MAX(pos_cash.SK_DPD_DEF),MEAN(pos_cash.CNT_INSTALMENT),...,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,PRODUCT_PLACE,PRODUCT_RANK,PRODUCT_INTEREST,YIELD_GROUP,TARGET
0,1373057,303930,10,9.0,9.0,1.0,3.666667,0.0,0.0,9.000000,...,-1910.0,-1580.0,-1670.0,-1663.0,0.0,other,NaN,1.0,3.0,0.0
1,1748125,380567,6,12.0,12.0,1.0,9.000000,0.0,0.0,10.833333,...,-214.0,116.0,-94.0,-86.0,1.0,x-sell,2.0,NaN,2.0,0.0
2,1079126,193976,7,12.0,11.0,NaN,3.000000,0.0,0.0,12.000000,...,-236.0,94.0,365243.0,365243.0,1.0,x-sell,2.0,NaN,2.0,0.0
3,1156350,445184,6,5.0,5.0,NaN,2.000000,0.0,0.0,5.000000,...,-2783.0,-2663.0,-2663.0,-2656.0,1.0,mobile,NaN,1.0,1.0,0.0
4,1381718,158094,6,4.0,4.0,NaN,2.750000,0.0,0.0,4.000000,...,-2872.0,-2782.0,-2782.0,-2734.0,0.0,household,NaN,0.0,2.0,0.0


In [11]:
app_pos_cash_agg = app_pos_cash_agg[app_pos_cash_agg['TARGET'].notna()]

In [20]:
app_pos_cash_agg.head()

,SK_ID_PREV,SK_ID_CURR,COUNT(pos_cash),MAX(pos_cash.CNT_INSTALMENT),MAX(pos_cash.CNT_INSTALMENT_FUTURE),MAX(pos_cash.RATIO_DPD),MAX(pos_cash.RATIO_Term_Payed_Left),MAX(pos_cash.SK_DPD),MAX(pos_cash.SK_DPD_DEF),MEAN(pos_cash.CNT_INSTALMENT),...,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,PRODUCT_PLACE,PRODUCT_RANK,PRODUCT_INTEREST,YIELD_GROUP,TARGET
0,1373057,303930,10,9.0,9.0,1.0,3.666667,0.0,0.0,9.000000,...,-1910.0,-1580.0,-1670.0,-1663.0,0.0,other,NaN,1.0,3.0,0.0
1,1748125,380567,6,12.0,12.0,1.0,9.000000,0.0,0.0,10.833333,...,-214.0,116.0,-94.0,-86.0,1.0,x-sell,2.0,NaN,2.0,0.0
2,1079126,193976,7,12.0,11.0,NaN,3.000000,0.0,0.0,12.000000,...,-236.0,94.0,365243.0,365243.0,1.0,x-sell,2.0,NaN,2.0,0.0
3,1156350,445184,6,5.0,5.0,NaN,2.000000,0.0,0.0,5.000000,...,-2783.0,-2663.0,-2663.0,-2656.0,1.0,mobile,NaN,1.0,1.0,0.0
4,1381718,158094,6,4.0,4.0,NaN,2.750000,0.0,0.0,4.000000,...,-2872.0,-2782.0,-2782.0,-2734.0,0.0,household,NaN,0.0,2.0,0.0


## Mutual information

In [12]:
# X = app_pos_cash_agg.drop(['TARGET', 'SK_ID_PREV', 'SK_ID_CURR'], axis=1)
# y = app_pos_cash_agg['TARGET']

# # Compute mutual information scores
# mi_scores = pd.DataFrame(mutual_info_classif(
#     X, y), index=X.columns, columns=['Mutual Information Gain'])
# mi_scores.sort_values(by='Mutual Information Gain',
#                       ascending=False, inplace=True)

# # Visualize the scores
# width = np.arange(len(mi_scores))
# ticks = list(mi_scores.index)

# plt.figure(dpi=100, figsize=(8, 5))
# plt.barh(width, mi_scores['Mutual Information Gain'])
# plt.yticks(width, ticks)
# plt.xlabel("Mutual Information Score")
# plt.title("Mutual Information Scores for Stroke")
# plt.show()